In [1]:

import numpy as np
import pandas as pd
# import psycopg2

from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib 
from sklearn.model_selection import train_test_split
# Importing the xgboost
import catboost
from catboost import CatBoostRegressor,Pool
import matplotlib.pyplot as plt
#import matplotlib.pyplot as plt

In [2]:


# ustawienie plików z parametemai modelu
model_path='c:\workspace\Python\polutions\models'
model_season='zima'
model_name='catboost_polution_model'
model_file = model_path+"\\"+model_season+'\\'+model_name+".cmb"
scaler_file=model_path+"\\" +model_name+"\\scaler"
mean_indexbytime_file = model_path+"\\" +model_name+"\\mean_byTime"
board_log =model_path+"\\" +model_name+"\\graph"

In [3]:
# dbuser='global_owner'
# dbpass='if(g=l)weatherpg'
# try:
#     conn = psycopg2.connect(dbname='weatherpg',user=dbuser,host='weatherpg.db.dv.interia.pl',password=dbpass)
# except:
#     print("I am unable to connect to the database")

In [4]:

#dataset_raw =pd.read_sql_query('select * from weather_owner.polution_table_data(119,204)',con=conn)
# dataset_raw =pd.read_sql_query(f'select * from  weather_owner.polution_table_data_season(119, 204,\'{model_season}\')',con=conn)

#dataset_raw =pd.read_sql_query('select * from weather_owner.polution_table_data_current(119,204)',con=conn)
dataset_raw =pd.read_csv('..\\data\\zima\csv\\4970_krakow.csv')
dataset_raw.set_index("data",inplace=True)


In [5]:
#dataset_raw.to_pickle('../picle/polution_raw.pickle')
#dataset_raw=pd.read_pickle('../picle/polution_raw.pickle')
#dataset_raw.read_pickle('../picle/polution_raw.pickle')

# abs(dataset_raw['prev_index_avg']-dataset_raw['index_avg']).mean()
# dataset_raw['index_avg'].std()

In [6]:
uniqe_data=pd.unique(dataset_raw.index)
uniqe_data_train, uniqe_data_test= train_test_split(uniqe_data, test_size = 0.2, random_state = 12)

In [7]:
col_list=['timeofday','is_weekend','windspeed6','dispel50','temp6','prev_index_avg','opady6','rhumid','wind_dir_ns','wind_dir_ew']
col_list_1=['timeofday','is_weekend','windspeed6','temp6','prev_index_avg','opady6','rhumid','dewpt']
col_list_2=['timeofday','is_weekend','dispel50','temp6','prev_index_avg','opady6','rhumid','dewpt']
col_list_3=['timeofday','is_weekend','temp6','prev_index_avg','opady6','rhumid','wind_dir_ns','wind_dir_ew','dewpt']


In [8]:
X =pd.DataFrame(dataset_raw, columns=col_list)
Y =pd.DataFrame(dataset_raw, columns=['index_avg'])
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 10)

#w naszym przypadku musimy brac całe dni do testowego zbioru bo jak bierzemy pojedyncze godziny to model się teoretycznieie uczy w nieskończoność
#a w praktyce to dopasowuje się do innych godzin tego dnia
# uniqe_data=pd.unique(dataset_raw.index)
# uniqe_data_train, uniqe_data_test= train_test_split(uniqe_data, test_size = 0.2, random_state = 12)
X_train=X[X.index.isin(uniqe_data_train)]
X_test =X[X.index.isin(uniqe_data_test)]
y_train=Y[Y.index.isin(uniqe_data_train)]
y_test =Y[Y.index.isin(uniqe_data_test)]

In [9]:
X1 =pd.DataFrame(dataset_raw, columns=col_list_1)
Y1 =pd.DataFrame(dataset_raw, columns=['index_avg'])
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 10)

#w naszym przypadku musimy brac całe dni do testowego zbioru bo jak bierzemy pojedyncze godziny to model się teoretycznieie uczy w nieskończoność
#a w praktyce to dopasowuje się do innych godzin tego dnia
X_train1=X1[X1.index.isin(uniqe_data_train)]
X_test1 =X1[X1.index.isin(uniqe_data_test)]
y_train1=Y1[Y1.index.isin(uniqe_data_train)]
y_test1=Y1[Y1.index.isin(uniqe_data_test)]


In [10]:
X2 =pd.DataFrame(dataset_raw, columns=col_list_2)
Y2 =pd.DataFrame(dataset_raw, columns=['index_avg'])
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 10)

#w naszym przypadku musimy brac całe dni do testowego zbioru bo jak bierzemy pojedyncze godziny to model się teoretycznieie uczy w nieskończoność
#a w praktyce to dopasowuje się do innych godzin tego dnia
X_train2=X2[X2.index.isin(uniqe_data_train)]
X_test2 =X2[X2.index.isin(uniqe_data_test)]
y_train2=Y2[Y2.index.isin(uniqe_data_train)]
y_test2=Y2[Y2.index.isin(uniqe_data_test)]


In [11]:
X3 =pd.DataFrame(dataset_raw, columns=col_list_3)
Y3 =pd.DataFrame(dataset_raw, columns=['index_avg'])
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 10)

#w naszym przypadku musimy brac całe dni do testowego zbioru bo jak bierzemy pojedyncze godziny to model się teoretycznieie uczy w nieskończoność
#a w praktyce to dopasowuje się do innych godzin tego dnia
X_train3=X3[X3.index.isin(uniqe_data_train)]
X_test3 =X3[X3.index.isin(uniqe_data_test)]
y_train3=Y3[Y3.index.isin(uniqe_data_train)]
y_test3=Y3[Y3.index.isin(uniqe_data_test)]


In [12]:
cat_features_index = [0,1]

#model = CatBoostRegressor(iterations=10000, learning_rate=0.01, depth=6,l2_leaf_reg=27)

model = CatBoostRegressor(iterations=7000, learning_rate=0.01, depth=6,l2_leaf_reg=27)
model1 = CatBoostRegressor(iterations=7000, learning_rate=0.01, depth=6,l2_leaf_reg=27)
model2 = CatBoostRegressor(iterations=7000, learning_rate=0.01, depth=6,l2_leaf_reg=27)
model3 = CatBoostRegressor(iterations=7000, learning_rate=0.01, depth=6,l2_leaf_reg=27)


In [13]:
#model.fit(X_train, y_train, cat_features_index,                 eval_set=[(X_test, y_test)],use_best_model=True,plot=True,silent=True                 )

#model.fit(X_train, y_train, cat_features_index,    eval_set=[(X_test, y_test)],use_best_model=True,plot=True,silent=True)
model.fit(X_train, y_train, cat_features_index,    eval_set=[(X_test, y_test)],use_best_model=True,silent=True)
model1.fit(X_train1, y_train1, cat_features_index,    eval_set=[(X_test1, y_test1)],use_best_model=True,silent=True)
model2.fit(X_train2, y_train2, cat_features_index,    eval_set=[(X_test2, y_test2)],use_best_model=True,silent=True)
model3.fit(X_train3, y_train3, cat_features_index,    eval_set=[(X_test3, y_test3)],use_best_model=True,silent=True)


In [14]:
model.save_model(model_path+"\\"+model_season+'\\'+model_name+".cmb", format="cbm")
model1.save_model(model_path+"\\"+model_season+'\\'+model_name+"_1.cmb", format="cbm")
model2.save_model(model_path+"\\"+model_season+'\\'+model_name+"_2.cmb", format="cbm")
model3.save_model(model_path+"\\"+model_season+'\\'+model_name+"_3.cmb", format="cbm")
#model.load_model('polution_catboost.cmb')


In [15]:
# model_file

In [19]:
model1.score(X_test1, y_test1)


14.359907554070778

In [21]:
pool = Pool(X_train3, y_train3)

fi = model3.get_feature_importance(pool,fstr_type='FeatureImportance')
#finteraction = model.get_feature_importance(pool,fstr_type='Interaction')
feature_importance=pd.DataFrame(fi,columns=['importance'])
feature_importance["futures"]=col_list_3


feature_importance.set_index("futures",inplace=True)
feature_importance.sort_values(by=["importance"],ascending =False,inplace=True )
feature_importance



,importance
futures,
prev_index_avg,20.866703
temp6,13.371391
rhumid,13.089764
wind_dir_ew,13.040492
wind_dir_ns,11.447908
opady6,9.106890
timeofday,8.897837
dewpt,7.756518
is_weekend,2.422495


In [26]:
pred=model.predict(X_test)
pred1=model1.predict(X_test1)
pred2=model2.predict(X_test2)
pred3=model3.predict(X_test3)

In [49]:
a=abs(y_test1['index_avg'].values-pred1)

In [50]:
a.mean()

11.508268566738458

In [51]:
model1.score(X_test1, y_test1)


14.359907554070778

In [52]:
a=abs(y_test2['index_avg'].values-pred2)

In [58]:
a.mean()

10.576167254892097

In [101]:
abs(y_test['index_avg'].values-pred).mean()

10.576167254892097

In [99]:
np.sqrt(pow(y_test['index_avg'].values-pred,2).mean())

13.382877612772763

In [96]:
model.score(X_test, y_test)


13.382877612772763

In [117]:
pred_aa=(pred1+pred2+pred)/3

In [120]:
np.sqrt(pow(y_test['index_avg'].values-pred_aa,2).mean())

13.665297873130875

In [118]:
abs(y_test['index_avg'].values-pred_aa).mean()

11.000908865969318

In [113]:
abs(y_test['index_avg'].values-pred2).mean()

11.426557642128248

In [114]:
abs(y_test['index_avg'].values-pred1).mean()

11.508268566738458

In [115]:
abs(y_test['index_avg'].values-pred3).mean()

13.409891347398633

In [116]:
abs(y_test['index_avg'].values-pred).mean()

10.576167254892097

Wniosek:
Suma słabych regresorów jestgorsza niż silny<br>
choć suma jest lepsza ni każdy z nich
